# Lecture 6: Python for Intrument Control (1.5 Hours) #

### ABSTRACT ###

In this lecture we will see how we can use Python to communicate with scientific instruments. First we will look at two packages that help add unit support to Python. Then we will revisit classes and how they can be usefull for desiging python interfaces to communicate to devices. We will then work on designing a driver for a demo instrument provided in the [git repo](https://github.com/QuinnPhys/PythonWorkshop-science) for this project.

## Unitful Computing with python-quantities and Pint (15 Minutes)##

Before getting to acutally controling instruments, we'll briefly mention how to use Python to perform numerical computations in a *unitful* way. We'll demonstrate using python-quantities, which provides a *quantity* data type to represent arrays with associated units (and is how units are handled in IntrumentKit). 

For now, let's go on and import python-quantities.

In [1]:
import numpy as np
import quantities as pq

We can now create quantities by specifying both a magnitude and a unit string.

In [2]:
velocity = pq.Quantity(2, 'm / s')
velocity

array(2) * m/s

Units can also be applied by arithmetically multiplying them with an array from numpy or just a python list. The following two ways of adding units are equvalent:

In [3]:
vader = np.array([1,2,3]) * pq.N
luke = [1,2,3] * pq.N
print(vader) 
print(luke)

[ 1.  2.  3.] N
[ 1.  2.  3.] N


Arithmetic operations on quantities respect units.

In [4]:
acceleration = velocity / pq.second
acceleration

array(2.0) * m/s**2

Importantly, operations on quantities must be correct for dimensions, but may use different units.

In [5]:
pq.Quantity(1, 'meter') + pq.Quantity(1, 'foot')

array(1.3048) * m

Now, you may ask what happens when we use dimensionally incorrect units. Look at the below raised error:

In [6]:
pq.Quantity(1, 'meter') + pq.Quantity(1, 'second')

ValueError: Unable to convert between units of "s" and "m"

This looks quite long, but there is basically two main places to look for what the error was: at the top ``ValueError`` and the bottom: ``ValueError: Unable to convert between units of "s" and "m"``. The text in the middle is just tracking where the fuctions were called from and is often not needed to troubleshoot the problem. We can use python statements ``try:`` and ``except`` to help us make errors more readable.

In [7]:
try:
    pq.Quantity(1, 'meter') + pq.Quantity(1, 'second')
except ValueError as ex:
    print(ex)

Unable to convert between units of "s" and "m"


Quanities also support transforming between different unit systems. You can do this in one of two ways, you can directly set the atribute ``.units`` of the object or use the rescale method. 

*NB: if you directly set the atribute by spectifying a named module variable (``pq.hour``) you must use the singular version of the name of the unit (not ``pq.hours``). If you are specifying a unit as a strin

In [8]:
love = pq.Quantity(525600.0, 'minutes')
love.units = pq.hour
print(love)
love.rescale('days')

8760.0 h


array(365.0) * d

Such conversions are also checked for consistent dimensions.

In [9]:
try:
    acceleration.rescale('kg')
except ValueError as ex:
    print(ex)

Unable to convert between units of "m/s**2" and "kg"


Finally, python-quantities also provides a range of useful physical constants that we can use to quickly construct quantities. 

In [10]:
Sz = pq.constants.hbar * np.array([[1, 0], [0, -1]]) / 2
H = pq.constants.gamma_p * pq.Quantity(7, 'tesla') * Sz
print(H)

[[ 3.5  0. ]
 [ 0.  -3.5]] gamma_p*(h/(2*pi))*T


To make it easier to compare values, you can use the atribute simplified to convert all the units to the defaults for that dimension. 

In [11]:
print(H.simplified)
(H.simplified / (2 * np.pi * pq.constants.hbar)).simplified.rescale('MHz')

[[  9.87424663e-26   0.00000000e+00]
 [  0.00000000e+00  -9.87424663e-26]] kg*m**2/s**2


array([[ 149.02118732,    0.        ],
       [   0.        , -149.02118732]]) * MHz

### Pint units package ###

As we might expect, there is more than one option for a packackage in python that can help us handle units. [Pint](https://pint.readthedocs.io/en/0.7.2/) is chronologically a newer package, but with a bit more features than quantities. 

First, let's check to see if the package is installed. Try running: 
```bash 
pip show pint
```
in your shell to see if pip knows about the package. If it returns nothing, then you do not have it installed and you should run:
```bash 
pip install pint
```
If if does print out some stuff to the screen, then it will show you where it has the package installed. Let's look at some examples of using the Pint package. 

Pints has this concept of a unit registry, that is a list of all the supported units and their relationship. 

In [12]:
import pint 
ureg = pint.UnitRegistry()

To use these units you simply multiply the magnitude of the value by an instance of a unit object defined by the unit registry. The variable ``time`` then becomes an instance of a quantitiy object:

In [13]:
time = 8.0 * ureg.second
print(time)
type(time)

8.0 second


pint.unit.build_quantity_class.<locals>.Quantity

You can also provide the units as a string that Pint will parse:

In [14]:
time = 8.0 * ureg('seconds')

Being an instance of a quantity type means that we can access atributes of ``time``:

In [15]:
print(time.magnitude)
print(time.units)
print(time.dimensionality)

8.0
second
[time]


We can look at how these ``Quantity`` objects are represented internaly:

In [16]:
print(repr(time))

<Quantity(8.0, 'second')>


This then allows us to be more direct about creating an instance of a ``Quantity``:

In [17]:
distance = ureg.Quantity(3, 'feet')
print(distance)

3 foot


We can perform arithmetic operations with the ``Quantity`` objects with no special syntax:

In [18]:
speed = np.absolute( distance / time )
print(speed)

0.375 foot / second


We can easily convert quantities with the ``.to`` method, which generates a new object with the new units. Else to change the units associated with the object ``speed`` we can ise the ``.ito`` method.

In [19]:
speed.to(ureg.km / ureg.hour)
print(speed)
speed.ito(ureg.km / ureg.hour)
print(speed)

0.375 foot / second
0.41147999999999996 kilometer / hour


Similar to quantities, Pint will warn when a particular unit conversion is not possible.

In [20]:
try:
    speed.to(ureg.N)
except pint.errors.DimensionalityError as ex:
    print(ex)

Cannot convert from 'kilometer / hour' ([length] / [time]) to 'newton' ([length] * [mass] / [time] ** 2)


Lastly, if we want to look up what units are defined in the current registry, we can look at the different unit systems the registry knows about. By default calling ``ureg = UnitRegistry()`` makes units from all systems available.

In [21]:
dir(ureg.sys)

['US', 'cgs', 'imperial', 'mks']

To see what units are defined in a specific unit system just look at the atributes of the system:

In [22]:
dir(ureg.sys.imperial)

['UK_hundredweight',
 'UK_ton',
 'acre_foot',
 'cubic_foot',
 'cubic_inch',
 'cubic_yard',
 'drachm',
 'foot',
 'grain',
 'imperial_barrel',
 'imperial_bushel',
 'imperial_cup',
 'imperial_fluid_drachm',
 'imperial_fluid_ounce',
 'imperial_gallon',
 'imperial_gill',
 'imperial_peck',
 'imperial_pint',
 'imperial_quart',
 'inch',
 'long_hunderweight',
 'long_ton',
 'mile',
 'ounce',
 'pound',
 'quarter',
 'short_hunderdweight',
 'short_ton',
 'square_foot',
 'square_inch',
 'square_mile',
 'square_yard',
 'stone',
 'yard']

## More on class inheritance (25 Minutes) ##

Recall that in Python all types are functions which return values of that type. Classes in python are a way of defining new types. Let's look a little more in-depth at classes and how they can be useful to us for communicating with instruments.

We will use this ``ExampleClass`` to explore our understanding of classes. It inherits the attributes of the ``object`` class which is the most basic class one can inherit from. Unlike the previous examples we are specifing an ``__init__`` definition. This function is called when an instance of the class is created and allows us to specify input arguments. 

In [23]:
class ExampleClass(object):
    '''
    An example of a basic class with attributes and methods.
    '''
    def __init__(self, name, value):
        '''
        Initialize the class, set the name, and value for the exponent method.
        '''
        self.name = name
        self.value = value
    
    tally = 0
    
    def test_exponent(self, x):
        self.tally = self.tally + 1
        return x ** self.value
    def say_name(self):
        print('My name is {name}. I raise thing to the power of {power}.'.format(name = self.name, power = self.value))
        

You might notice the text inclosed with '''. This is called a ``Docstring`` and is basically documentation attached to that class. If you try running the below line, you can see that this text is then displayed in the pop-up.

In [24]:
?ExampleClass

We can then create an instance of this class by choosing a name and an exponent value. We can also check on the value of the attribute ``tally``:

In [25]:
cls = ExampleClass('Bob', 3)
cls.say_name()
print(cls.test_exponent(3))
print(cls.tally)

My name is Bob. I raise thing to the power of 3.
27
1


We can check to see if  the atribute ``tally`` is keeping track of how many times we have called the ``test_exponent`` method.

In [26]:
print(cls.test_exponent(4))
print(cls.tally)

64
2


If we want to look to see what attributes that ``ExampleClass`` is inheriting from ``object`` we can use the ``dir`` function to list atributes:

In [27]:
dir(object)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

Inheriting these atributes means that they are also valid atributes of ``ExampleClass``. For example ``__sizeof__`` is defined for an instance of our example class ``cls`` and returns the internal size in bytes for the given object:

In [28]:
cls.__sizeof__()

32

Having this inheriting structure for classes makes it easy to share definitions for methods, which can reduce copy/pasting of code and make things more readable. We are going to explore this idea further now by looking at some "general" or "abstract" classes that could describe an instrument we might want to control with python. Then to control a specific model of that instrument we can write a new class that inherits methods and atributes that we have already specified as being a part of an abstract version of that instrument. 

Python packages that you might find for instrument control will work similarly, you use the package both for the template classes that they define for instruments, as well as haveing specific classes that are writen to talk to specific devices. 

## Let's build up an Instrument and talk to it! (50 Minutes) ##

For the purposes of this lecture we will use the ``InstrumentKit`` python package to write a class that can talk to a fake instrument. **TODO: instrument install instructions**

Consider the following situation: your experiment needs constant babysitting to set values and make sure that it functions correctly. You decide that to make your life simpler you will write a program that can set all the measurements needed and save the output data. The device you need to control is a four channel DC power supply. Opening the manual you find the following table for what commands you can send to the device:

| SPCI command              | Example usage                                                      | Options                                                                                 | Description                                           |
|---------------------------|--------------------------------------------------------------------|-----------------------------------------------------------------------------------------|-------------------------------------------------------|
| ``*IDN?``                | Query return:<br>``EPQIS16 Demonstration Instrument``              |                                                                                         | Find out what the name of the instument is.           |
| ``VOLTS? <ch>``          | ``VOLTS? 0``<br>``VOLTS? 1``<br>Query return:<br>``{value in mV}`` | ``<ch> = {0,1,2,3}``<br>Default: ``0``                                                      | Check what the ouput voltage is set to  (returns mV). |
| ``VOLTS  <ch> <val>``    | ``VOLTS 0 314``<br>``VOLTS 1 159``                               | ``<ch> = {0,1,2,3}``<br>Default: ``0``<br>``<val> = voltage value in mV``<br>Default: ``0`` | Set the voltage output (provided value is mV).        |
| ``ENABLE <ch> <state>``  | ``ENABLE 0 OFF``<br>``ENABLE 1 ON``                              | ``<ch> = {0,1,2,3}``<br>Default: ``0``<br>``<state> = {ON,OFF}``<br>Default: ``OFF``        | Toggle the voltage channel output on/off.             |     
| ``ENABLE? <ch>``         | ``ENABLE? 0``<br>``ENABLE? 1``<br> Query return:<br>``{ON, OFF}``  | ``<ch> = {0,1,2,3}``<br>Default: ``0``                                                      | Check the ouput status of a voltage channel.          | 

We can do some inital debugging of the communication channel if we just treat the instrument as a generic SCPI intrument from the ``InstrumentKit`` package. Let's start by importing as ik:

In [33]:
import instruments as ik

In [39]:
print(ik.generic_scpi.SCPIInstrument.__doc__)


    Base class for all SCPI-compliant instruments. Inherits from
    from `~instruments.Instrument`.

    This class does not implement any instrument-specific communication
    commands. What it does add is several of the generic SCPI star commands.
    This includes commands such as ``*IDN?``, ``*OPC?``, and ``*RST``.

    Example usage:

    >>> import instruments as ik
    >>> inst = ik.generic_scpi.SCPIInstrument.open_tcpip('192.168.0.2', 8888)
    >>> print(inst.name)
    


In [30]:
ins = ik.generic_scpi.SCPIInstrument.open_tcpip("localhost",8042)

In [31]:
type(ins)

instruments.generic_scpi.scpi_instrument.SCPIInstrument

In [32]:
ins.channel[2].voltage = Quantities(43,"millivolts")

NameError: name 'Quantities' is not defined

In [ ]:
ins.query("VOLTS? 2")

In [ ]:
ins.sendcmd("VOLTS 2 43")

In [ ]:
ins.query("ENABLE? 2")

In [ ]:
ins.sendcmd("ENABLE 1 ON")

In [ ]:
ins.query("*IDN?")

## Lots of options of starting points, don't reinvent the wheel! ## 

We all have no more valuable resource than our own time, so there is little point to reimplimenting functionality that exsists in other packages that will work in your Python workflow. As ususal, google your instrument to see if there is a python package that already supports it (or if the manufacuter can supply control code samples!). Here are some popular packages that together cover a lot of instruments. 

- [InstrumentKit](http://instrumentkit.readthedocs.io/en/latest/index.html)
- [Instrumental](http://instrumental-lib.readthedocs.io/en/latest/)
- [QuDi](http://qosvn.physik.uni-ulm.de/trac/qudi)  
- [Python IVI](http://alexforencich.com/wiki/en/python-ivi/readme)